In [ ]:
pyspark --master=local[1] --jars /root/library/commons-httpclient-3.0.1.jar,/root/library/spark-sql-kafka-0-10_2.12-3.1.1.jar,/root/library/commons-pool2-2.10.0.jar,/root/library/kafka-clients-2.8.0.jar,/root/library/spark-token-provider-kafka-0-10_2.12-3.1.1.jar

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark import SparkContext
from pyspark.conf import SparkConf


In [ ]:
sc.addFile("/root/mymodel/mymodel.py")
import mymodel

In [ ]:
df_raw = spark \
  .readStream \
  .format('kafka') \
  .option('kafka.bootstrap.servers', 'localhost:9092') \
  .option("startingOffsets", "earliest") \
  .option('subscribe', 'inchat') \
  .load()


In [ ]:
json_data = df_raw.selectExpr('CAST(value AS STRING) as value')